Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [36]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [55]:
import pandas as pd
import numpy as np
# import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 thousands=',', engine='python')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [56]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [57]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [58]:
# Filter the data for One Family Dwellings
df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

# Filter the sale price between $100K and $2 million
df = df[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)]

# Drop the EASE-MENT and APARTMENT_NUMBER columns (full of NaNs)
df = df.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'])

# Convert LAND_SQUARE_FEET to numerical
df['LAND_SQUARE_FEET'] = pd.to_numeric(df['LAND_SQUARE_FEET'])

In [59]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,1,A5,620000,01/02/2019


In [60]:
df.shape

(3164, 19)

### Do train-test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [61]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].dt.month.value_counts()

1    952
3    802
2    763
4    647
Name: SALE_DATE, dtype: int64

In [62]:
train = df[df['SALE_DATE'].dt.month < 4]
test  = df[df['SALE_DATE'].dt.month == 4]
train.shape, test.shape

((2517, 19), (647, 19))

In [63]:
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2517,1,01 ONE FAMILY DWELLINGS,2517,NaN,NaN
TAX_CLASS_AT_PRESENT,2517,2,1,2486,NaN,NaN
BOROUGH,2517,5,4,1209,NaN,NaN
NEIGHBORHOOD,2517,6,OTHER,2390,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921,NaN,NaN
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921,NaN,NaN
SALE_DATE,2517,68,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-03-30 00:00:00
ADDRESS,2517,2507,33 BAILEY PLACE,2,NaN,NaN


In [64]:
# Drop the non-numeric columns with high cardinality and target column
target = 'SALE_PRICE'
max_cardinality = 50  # arbitrary cut-off

high_cardinality = [col for col in train.select_dtypes(exclude=np.number)
                    if train[col].nunique() > max_cardinality]

features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
X_test = test[features]
y_train = train[target]
y_test = test[target]

### Do one-hot encoding of categorical features.

In [65]:
import category_encoders as ce

In [66]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [67]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,...,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,...,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
BOROUGH_3,1.000000,-0.418527,-0.141981,-0.260592,-0.008679,-0.003803,-0.077849,0.137608,-0.035898,0.158991,...,-0.103066,-0.003347,-0.071989,-0.111871,-0.027189,0.059652,0.157202,-0.033707,-0.048612,0.045796
BOROUGH_4,-0.418527,1.000000,-0.313564,-0.575514,-0.019167,-0.163476,0.186008,-0.057593,0.085772,-0.066542,...,0.174356,-0.165262,0.107784,0.079603,-0.034254,0.008157,-0.071902,-0.053777,0.044051,-0.019167
BOROUGH_2,-0.141981,-0.313564,1.000000,-0.195238,-0.006502,0.075183,-0.058325,-0.019538,-0.026895,-0.022574,...,-0.018328,0.060916,-0.037190,-0.036273,-0.007223,0.002731,-0.006271,-0.025253,0.085774,-0.006502
BOROUGH_5,-0.260592,-0.575514,-0.195238,1.000000,-0.011934,0.137991,-0.107050,-0.035859,-0.049363,-0.041432,...,-0.099131,0.149992,-0.037385,0.027322,0.066357,-0.060499,-0.044769,0.105921,-0.066846,-0.011934
BOROUGH_1,-0.008679,-0.019167,-0.006502,-0.011934,1.000000,0.004596,-0.003565,-0.001194,-0.001644,-0.001380,...,-0.015145,-0.013372,-0.003297,-0.008871,-0.002468,-0.002501,-0.001491,-0.001544,-0.002226,-0.000397
NEIGHBORHOOD_OTHER,-0.003803,-0.163476,0.075183,0.137991,0.004596,1.000000,-0.775780,-0.259869,-0.357727,-0.300251,...,0.069602,-0.029792,-0.074641,0.014612,-0.045885,-0.044561,-0.080382,0.017849,0.025741,0.004596
NEIGHBORHOOD_FLUSHING-NORTH,-0.077849,0.186008,-0.058325,-0.107050,-0.003565,-0.775780,1.000000,-0.010713,-0.014747,-0.012377,...,-0.021615,-0.010916,0.112860,0.031533,-0.022141,0.014691,-0.013374,-0.013847,-0.019970,-0.003565
NEIGHBORHOOD_BEDFORD STUYVESANT,0.137608,-0.057593,-0.019538,-0.035859,-0.001194,-0.259869,-0.010713,1.000000,-0.004940,-0.004146,...,-0.045506,0.002984,-0.009906,-0.026656,-0.007417,-0.007515,0.353504,-0.004638,-0.006689,-0.001194
NEIGHBORHOOD_FOREST HILLS,-0.035898,0.085772,-0.026895,-0.049363,-0.001644,-0.357727,-0.014747,-0.004940,1.000000,-0.005707,...,-0.032431,0.039059,-0.013637,-0.036693,0.188688,-0.010345,-0.006167,-0.006385,-0.009208,-0.001644
NEIGHBORHOOD_BOROUGH PARK,0.158991,-0.066542,-0.022574,-0.041432,-0.001380,-0.300251,-0.012377,-0.004146,-0.005707,1.000000,...,-0.040603,0.003448,-0.011446,-0.030798,-0.008569,0.038016,-0.005176,-0.005359,-0.007729,-0.001380
